###### A ideia aqui é apresentar uma proposta de resolução de um problema de otimização, especificamente para a busca em grafo, tomando como  exemplo o problema do caixeiro viajante.
###### Esse tema é amplamente abordado em problemas de logística por se tratar da tentativa de se obter a melhor solução para minimizar a distância de uma rota, esta que passa por vários pontos, dado que este ponto inicial também é o ponto final. O que torna a situação
###### muito importante no mundo real para casos que envolvam entregas ou deslocamento de pessoas (correios, delivery, transporte em geral).  



##### Para o problema vamos considerar que cada cidade só pode ser visitada uma única vez e que todas se comunicam diretamente entre si.

###### Primeiramente vamos importar as bibliotecas e métodos necessários

In [1]:
import random
import numpy as np 
from deap import algorithms, base, creator, tools

###### Definindo as cidades, suas localizações no plano e colocando cada combinação das cidadees numa matriz
###### para facilitar os cálculos das distências 

In [2]:
cities = {
    0 : (0, 0),
    1: (1, 5),
    2: (5, 2),
    3: (3, 6),
    4: (8, 3),
    5: (10, 10),
    6: (12, 6),
    7: (9, 0)
}

num_cities = len(cities)
distances = np.zeros((num_cities, num_cities))

###### Calculando as distâncias entre cada par de cidades na matriz distances

In [3]:
for i in range(num_cities):
    for j in range(num_cities):
        x1, y1 = cities[i]
        x2, y2 = cities[j]
        distances[i][j] = np.sqrt((x2 - x1)**2 + (y2 - y1)**2)

###### Definindo uma função para a criação dos indivíduos

In [4]:
def create_individual():
    individual = list(cities.keys())
    random.shuffle(individual)
    return individual

###### A função abaixo recebe cada indivíduo criado e calcula as distâncias entre as cidades

In [5]:
def evaluate_individual(individual):
    distance = 0
    for i in range(num_cities - 1):
        city_idx1 = individual[i]
        city_idx2 = individual[i + 1]
        distance += distances[city_idx1][city_idx2]
    return distance,

###### Abaixo definimos os tipos e a função de aptidão

In [6]:
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

###### Definindo as toolboxes 

In [7]:
toolbox = base.Toolbox()
toolbox.register("individual", tools.initIterate, creator.Individual, create_individual)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("evaluate", evaluate_individual)
toolbox.register("mate", tools.cxPartialyMatched)
toolbox.register("mutate", tools.mutShuffleIndexes, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)


###### Definindo os parâmetros da população

In [8]:
population_size = 100
num_generations = 100
cx_prob = 0.8
mut_prob = 0.2

###### Criando a população inicial

In [9]:
population = toolbox.population(n=population_size)

###### Execução do algorítmo genético 

In [10]:
for generation in range(num_generations):
    offspring = algorithms.varAnd(population, toolbox, cxpb=cx_prob, mutpb=mut_prob)
    fitnesses = toolbox.map(toolbox.evaluate, offspring)
    for ind, fit in zip(offspring, fitnesses):
        ind.fitness.values = fit
    population = toolbox.select(offspring, k=len(population))

###### Obtendo a melhor solução encontrada 

In [11]:
best_solution = tools.selBest(population, k=1)[0]
best_distance = evaluate_individual(best_solution)[0]
print("Melhor solução encontrada: ", best_solution)
print("Distância mínima percorrida: ", best_distance.round(2))

Melhor solução encontrada:  [0, 1, 3, 2, 7, 4, 6, 5]
Distância mínima percorrida:  28.91
